In [37]:
# Reference webiste for the model: https://towardsdatascience.com/time-series-forecasting-with-recurrent-neural-networks-74674e289816

#Imports
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from pathlib import Path
%matplotlib inline
import plotly.graph_objs as go

In [39]:
#Importing CSV
csvpath = Path("Resources/daily_QQQ.csv")
ETF = pd.read_csv(csvpath)
ETF.head()


,timestamp,open,high,low,close,volume
0,1/22/2021,325.29,326.46,324.710,325.42,21609444
1,1/21/2021,325.15,327.13,323.730,326.36,25316935
2,1/20/2021,320.14,324.74,316.509,323.77,30728096
3,1/19/2021,314.25,316.93,313.120,316.41,24537049
4,1/15/2021,314.13,315.12,310.580,311.86,35118718


In [40]:
# Removing all columns but close_price
ETF.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)
ETF.head()

,timestamp,close
0,1/22/2021,325.42
1,1/21/2021,326.36
2,1/20/2021,323.77
3,1/19/2021,316.41
4,1/15/2021,311.86


In [41]:
#Checking For Nulls
ETF.isnull().sum()

timestamp    0
close        0
dtype: int64

In [42]:
#Establishing test and train data for Closing price as well as the Date
close_data = ETF['close'].values
close_data = close_data.reshape((-1,1))

In [43]:
split_percent = 0.7
split = int(split_percent*len(close_data))

In [44]:
close_train = close_data[:split]
close_test = close_data[split:]
close_train

array([[325.42],
       [326.36],
       [323.77],
       ...,
       [ 41.21],
       [ 41.11],
       [ 41.55]])

In [46]:
date_data = ETF['timestamp']
date_train = ETF['timestamp'][:split]
date_test = ETF['timestamp'][split:]
print(len(close_train))
print(len(close_test))

3738
1603


In [47]:
# setting the lookback, shape and 
look_back = 45

shape = (look_back,1)
# Reformat the data set to dataset of windows over a period of time.
#TimeseriesGenerator(Data, Target, sequence_length, batch)

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [48]:
#Build LSTM Model
model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape= shape,
))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')


In [49]:
# Show the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10)                480       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________


In [50]:
# Train the model
#used fit_generator because we are looking to generate new predicted data points 
num_epochs = 200
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/200
185/185 [==============================] - 4s 12ms/step - loss: 16382.9506
Epoch 2/200
185/185 [==============================] - 2s 11ms/step - loss: 22882.7771
Epoch 3/200
185/185 [==============================] - 2s 11ms/step - loss: 8903.3276
Epoch 4/200
185/185 [==============================] - 2s 13ms/step - loss: 5910.9942
Epoch 5/200
185/185 [==============================] - 3s 15ms/step - loss: 4594.1435
Epoch 6/200
185/185 [==============================] - 2s 13ms/step - loss: 3887.0115
Epoch 7/200
185/185 [==============================] - 2s 13ms/step - loss: 2557.0388
Epoch 8/200
185/185 [==============================] - 2s 13ms/step - loss: 1618.9511
Epoch 9/200
185/185 [==============================] - 2s 13ms/step - loss: 1038.1418
Epoch 10/200
185/185 [==============================] - 2s 13ms/step - loss: 740.9834
Epoch 11/200
185/185 [==============================] - 3s 15ms/step - loss: 471.4939
Epoch 12/200
185/185 [==============================

185/185 [==============================] - 2s 13ms/step - loss: 31.6247
Epoch 97/200
185/185 [==============================] - 2s 13ms/step - loss: 17.0747
Epoch 98/200
185/185 [==============================] - 3s 14ms/step - loss: 24.6203
Epoch 99/200
185/185 [==============================] - 2s 13ms/step - loss: 20.4755
Epoch 100/200
185/185 [==============================] - 2s 13ms/step - loss: 35.8929
Epoch 101/200
185/185 [==============================] - 2s 13ms/step - loss: 24.1133
Epoch 102/200
185/185 [==============================] - 2s 13ms/step - loss: 25.4130
Epoch 103/200
185/185 [==============================] - 2s 13ms/step - loss: 26.9766
Epoch 104/200
185/185 [==============================] - 2s 13ms/step - loss: 23.9110
Epoch 105/200
185/185 [==============================] - 3s 15ms/step - loss: 27.1128
Epoch 106/200
185/185 [==============================] - 2s 13ms/step - loss: 25.7817
Epoch 107/200
185/185 [==============================] - 3s 16ms/step -

185/185 [==============================] - 2s 13ms/step - loss: 4.0859
Epoch 191/200
185/185 [==============================] - 2s 13ms/step - loss: 5.6466
Epoch 192/200
185/185 [==============================] - 3s 14ms/step - loss: 5.5512
Epoch 193/200
185/185 [==============================] - 3s 14ms/step - loss: 3.2487
Epoch 194/200
185/185 [==============================] - 2s 13ms/step - loss: 4.4010
Epoch 195/200
185/185 [==============================] - 2s 13ms/step - loss: 4.0123
Epoch 196/200
185/185 [==============================] - 3s 13ms/step - loss: 3.2610
Epoch 197/200
185/185 [==============================] - 3s 15ms/step - loss: 3.7309
Epoch 198/200
185/185 [==============================] - 3s 14ms/step - loss: 4.6365
Epoch 199/200
185/185 [==============================] - 3s 15ms/step - loss: 3.9087
Epoch 200/200
185/185 [==============================] - 3s 15ms/step - loss: 3.5684


In [55]:
#Make predictions for futre prices
prediction = model.predict_generator(test_generator)

#Test model predictions for accuracy through plottting
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

#PLot for training prices
trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
# PLotting our predicted prices
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
#plotting the actual prices for the period of time that was predicted
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
#Plot features and labels
layout = go.Layout(
    title = "QQQ",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
#Show plot
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [53]:
#Predicting future prices

close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = ETF['timestamp'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 90
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [54]:
forecast

array([130.8       , 126.37072754, 131.66734314, 131.50167847,
       135.90713501, 138.2288208 , 140.70648193, 142.95347595,
       144.49681091, 146.34480286, 147.29627991, 148.09153748,
       148.66459656, 148.96720886, 149.10534668, 149.32637024,
       149.37025452, 149.33241272, 149.25378418, 149.24008179,
       149.17819214, 149.02114868, 148.72438049, 148.30281067,
       147.88961792, 147.68577576, 147.56257629, 147.38673401,
       147.05036926, 146.74356079, 146.54058838, 146.28671265,
       145.94741821, 145.70256042, 145.39241028, 145.13011169,
       144.86174011, 144.58834839, 144.33351135, 144.13092041,
       143.92579651, 143.74137878, 143.57678223, 143.42588806,
       143.30529785, 143.19303894, 143.14122009, 143.10983276,
       143.13415527, 143.19110107, 143.28059387, 143.39810181,
       143.5287323 , 143.67811584, 143.82371521, 143.96278381,
       144.09040833, 144.20031738, 144.29064941, 144.36558533,
       144.42152405, 144.45852661, 144.47802734, 144.48

In [31]:
#making a dataframe
predicted_prices = pd.DataFrame({'Date': forecast_dates, 'PZA_close_price': list(forecast)}, columns=['Date', 'PZA_close_price'])
predicted_prices.head()

,Date,PZA_close_price
0,2021-01-13 00:00:00-05:00,27.090000
1,2021-01-14 00:00:00-05:00,27.168526
2,2021-01-15 00:00:00-05:00,27.249393
3,2021-01-16 00:00:00-05:00,27.327137
4,2021-01-17 00:00:00-05:00,27.401978


In [32]:
#export to excel
predicted_prices.to_csv(r'Resources/PZA_forecast.csv')

In [34]:
# Combine all forecasted prices into a single DF
LQDpath = Path("Resources/LQD_forecast.csv")
LQD = pd.read_csv(LQDpath)
#LQD.set_index(LQD['Date'], inplace=True)

PZApath = Path("Resources/PZA_forecast.csv")
PZA = pd.read_csv(PZApath)
PZA = PZA['PZA_close_price']
#PZA.set_index(PZA['Date'], inplace=True)

USOpath = Path("Resources/USO_forecast.csv")
USO = pd.read_csv(USOpath)
USO = USO['USO_close_price']
#USO.set_index(USO['Date'], inplace=True)

TLTpath = Path("Resources/TLT_forecast.csv")
TLT = pd.read_csv(TLTpath)
TLT = TLT['TLT_close_price']
#TLT.set_index(TLT['Date'], inplace=True)

QQQpath = Path("Resources/QQQ_forecast.csv")
QQQ = pd.read_csv(QQQpath)
QQQ = QQQ['QQQ_close_price']
#QQQ.set_index(QQQ['Date'], inplace=True)

VMBSpath = Path("Resources/VMBS_forecast.csv")
VMBS = pd.read_csv(VMBSpath)
VMBS = VMBS['VMBS_close_price']
#VMBS.set_index(VMBS['Date'], inplace=True)

VNQpath = Path("Resources/VNQ_forecast.csv")
VNQ = pd.read_csv(VNQpath)
VNQ = VNQ['VNQ_close_price']

PZA.head()

0    27.090000
1    27.168526
2    27.249393
3    27.327137
4    27.401978
Name: PZA_close_price, dtype: float64

In [35]:
#Concatenate and clean the data
forecast_prices = pd.concat([LQD, PZA, USO, TLT, QQQ, VMBS, VNQ], axis='columns', join='inner')
forecast_prices = forecast_prices.drop(columns=["Unnamed: 0"])
forecast_prices.head()

,Date,LQD_close_price,PZA_close_price,USO_close_price,TLT_close_price,QQQ_close_price,VMBS_close_price,VNQ_close_price
0,2021-01-13 00:00:00-05:00,136.520000,27.090000,35.760000,152.360000,316.000000,54.010000,84.310000
1,2021-01-14 00:00:00-05:00,137.277298,27.168526,35.348991,154.565033,303.799408,54.105686,81.114426
2,2021-01-15 00:00:00-05:00,138.050400,27.249393,35.693897,155.340942,302.885742,54.160480,80.127251
3,2021-01-16 00:00:00-05:00,138.771729,27.327137,35.377987,157.228912,302.805603,54.233402,80.206505
4,2021-01-17 00:00:00-05:00,139.553513,27.401978,35.770760,157.639771,302.869537,54.292553,80.162140


In [36]:
#EXPORT EXCEL
forecast_prices.to_csv(r'Resources/forecast_prices.csv')